In [50]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings

In [9]:
label_dict = {0: '(단언) 진술하기', 1: '(지시) 충고/제안하기', 2: '(단언) 주장하기', 3: '(지시) 질문하기',
              4: '(지시) 부탁하기', 5: '(단언) 반박하기', 6: '(표현) 감사하기', 7: '(표현) 사과하기',
              8: '(지시) 명령/요구하기', 9: '턴토크 사인(관습적 반응)',
              10: '(언약) 약속하기(제3자와)/(개인적 수준)', 11: '(언약) 거절하기', 12: '(표현) 인사하기'}

In [10]:
from transformers import TextClassificationPipeline, BertTokenizerFast, TFBertForSequenceClassification
import os


# MODEL_NAME = 'fine-tuned-klue-bert-base'
# MODEL_SAVE_PATH = os.path.join("_model", MODEL_NAME) # change this to your preferred location

MODEL_SAVE_PATH = '/home/inter/chat_MBTI/intent_classifier'

# Load Fine-tuning model
tokenizer = BertTokenizerFast.from_pretrained(MODEL_SAVE_PATH)
model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=tokenizer,
    model=model,
    framework='tf',
    return_all_scores=True
)

Some layers from the model checkpoint at /home/inter/chat_MBTI/intent_classifier were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /home/inter/chat_MBTI/intent_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
Device set to use 0
/home/inter

In [11]:
max_seq_len = 100
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, padding='max_length')

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]

    score = np.argmax(model.predict(encoded_input)[0])

    print(label_dict[score])

In [12]:
while True :
  new_sentence = input('sentence > ')
  if new_sentence == '끝':
    break
  sentiment_predict(new_sentence)

1/1 [==============================] - 5s 5s/step
(표현) 인사하기
1/1 [==============================] - 0s 211ms/step
턴토크 사인(관습적 반응)


In [41]:
import numpy as np

def intent_predict(sentence):

  temp_li = []
  score_li = text_classifier(sentence)[0]
  for i in score_li:
    temp_li.append(i['score'])

  return label_dict[int(score_li[temp_li.index(max(temp_li))]['label'][6:])]

In [42]:
intent_predict('하이')

'(표현) 인사하기'